# Dev: `fs`

In [ ]:
import os
from pathlib import Path

## `read_text()`

In [ ]:
def read_text(path, encoding='utf-8'):    
    """Read a text file and return its content as a string.
    """
    path = Path(path)
    try:
        # Open the text file in read mode
        with open(path, 'r', encoding=encoding) as file:
            # Read the entire file content into a string
            text_content = file.read()
            return text_content
    except FileNotFoundError:
        print(f"File was not found: `{path}`")
    except Exception:
        raise

In [44]:
read_text("not-exist.txt")

File was not found: `not-exist.txt`


In [46]:
read_text("text/hello.txt")
read_text(Path("text/hello.txt"))

'Hello world\nThis is secound line\n'

### HowTo

In [ ]:
with open("text/hello.txt", 'r', encoding='utf-8') as file:
    res1 = file.read()
    
res1

'Hello world\nThis is secound line\n'

In [6]:
with open("text/hello.txt", 'r', encoding='utf-8') as file:
    res2 = file.readlines()
    
res2

['Hello world\n', 'This is secound line\n']

## `write_text()`

In [101]:
def write_text(text, path, encoding="utf-8", recurse = False, append = False):
    """Write a text string to a text file.
    """
    path = Path(path)
    try:
        # Create intermediate directories if they don't exist
        if recurse and not path.parent.exists():
            path.parent.mkdir(parents=True, exist_ok=True)
            print(f"Created intermediate dir: {path.parent}")
        
        # Open the text file to write or append content
        with open(path, "a" if append else "w", encoding=encoding) as file:
            file.write(text)
            if append:
                print(f"Text successfully append to {path}")
            else:
                print(f"Text successfully written to {path}")
        
    except (PermissionError, IsADirectoryError):
        print(f"Cannot write to file: `{path}`")
    except Exception:
        raise


In [96]:
write_text("hi", Path("text/hello-write.txt"), append=False)
write_text("hi2", Path("text/hello-write2.txt"), append=False)

Text successfully written to text/hello-write.txt
Text successfully written to text/hello-write2.txt


In [102]:
write_text("hi", Path("text/subdir2/hello-write.txt"), recurse=False)

FileNotFoundError: [Errno 2] No such file or directory: 'text/subdir2/hello-write.txt'

In [105]:
write_text("hi", Path("text/subdir/sub/hello-write.txt"), recurse=True)

Created intermediate dir: text/subdir/sub
Text successfully written to text/subdir/sub/hello-write.txt


## `read_text_dir()`

In [229]:
def read_text_dir(path, 
                  recurse: bool = True,
                  pattern: str = '*',
                  case_sensitive: bool | None = None,
                  encoding: str = "utf-8",
                  ):
    """Read all text files in a directory and return their contents in a dictionary.
    """
    path = Path(path)
    # Validate that the path exists and is a directory
    if not path.exists():
        raise FileNotFoundError(f"The path does not exist: '{path}'")
    if not path.is_dir():
        raise NotADirectoryError(f"The path is not a directory: '{path}'")
    
    # Get mapping of pattern
    if recurse: 
        path_map = path.rglob(pattern, case_sensitive = case_sensitive)
    else:
        path_map = path.glob(pattern, case_sensitive = case_sensitive)
        
    # Get the list of path that is files and not dotfile 
    path_list = [f for f in path_map if f.is_file() and not f.name.startswith(".")]
    path_name_list = [f.name for f in path_list]
    
    # Read files in to dictionary where keys are filenames 
    content_dict = {}
    
    for path_name, path in zip(path_name_list, path_list):
        try:
            content_dict[path_name] = read_text(path, encoding = encoding)
        except Exception as e:
            print(f"Error reading file: '{path_name}'\n {e}")
            continue

    # Sort by filename
    content_dict_sorted = dict(sorted(content_dict.items()))
    
    return content_dict_sorted
    

In [230]:
content_dict1 = read_text_dir("text", recurse=True, pattern="*.txt")
content_dict1

{'hello-write.txt': 'hi',
 'hello-write2.txt': 'hi2',
 'hello.txt': 'Hello world\nThis is secound line\n'}

In [171]:
Path('text/.dotfile').name.startswith(".")
Path('text/hello-write.txt').name.startswith(".")

False

In [158]:
[f for f in Path("text").rglob(pattern = "*.txt") if f.is_file()]

[PosixPath('text/hello-write2.txt'),
 PosixPath('text/hello-write.txt'),
 PosixPath('text/hello.txt'),
 PosixPath('text/subdir/hello-write.txt'),
 PosixPath('text/subdir/sub/hello-write.txt')]

In [ ]:
list(Path("text").rglob('*'))

[PosixPath('text/subdir'),
 PosixPath('text/hello-write2.txt'),
 PosixPath('text/hello-write.txt'),
 PosixPath('text/hello.txt'),
 PosixPath('text/subdir/sub'),
 PosixPath('text/subdir/hello-write.txt'),
 PosixPath('text/subdir/sub/hello-write.txt')]

In [133]:
Path('text/fake.txt').is_dir()

False

In [124]:
Path('text/hello-write2.txt').is_dir()
Path('text/hello-write2.txt').is_file()

True

In [123]:
Path("text").is_dir()
Path("text").is_file()

False

In [114]:
file_ls1 = [f for f in os.listdir(Path("text")) if not f.startswith('.')] 
file_ls1

['subdir', 'hello-write2.txt', 'hello-write.txt', 'hello.txt']

In [115]:
[Path("text") / file for file in file_ls1]

[PosixPath('text/subdir'),
 PosixPath('text/hello-write2.txt'),
 PosixPath('text/hello-write.txt'),
 PosixPath('text/hello.txt')]